In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 104)
pd.set_option('display.max_columns',104)
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
sns.set_style('whitegrid')
%matplotlib inline


In [ ]:
# Load the library with the CountVectorizer method
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer 
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_recall_fscore_support, precision_score, recall_score, f1_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Embedding,Dropout, Layer ,LSTM, SpatialDropout1D,Bidirectional,GlobalMaxPool1D, Input
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping 
import keras.backend as K
from tensorflow.keras.optimizers import Adam
import keras
import multiprocessing
nthreads = multiprocessing.cpu_count()
import spacy
nlp = spacy.load('en_core_web_sm')
import string
punctuations = string.punctuation
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [ ]:
df = pd.read_csv('/kaggle/input/bonding/hm_train.csv')
test = pd.read_csv('/kaggle/input/bonding/hm_test.csv')
df.tail(3)

In [ ]:
stop_words = set(stopwords.words("english"))
def cleanup_text(doc):
    #print(1)
    texts = []
    #doc = re.sub(r"\$\s[0-9]+", "moneyvalue", doc)        
    doc = nlp(doc, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']

    tokens = [tok for tok in tokens if tok not in stop_words and tok not in punctuations and not tok.isdigit()]
    tokens = ' '.join(tokens)

    texts.append(tokens)
    return pd.Series(texts)

In [ ]:
%%time
with multiprocessing.Manager().Pool(nthreads) as pool:
    res = pool.map_async(cleanup_text, df['cleaned_hm'].values)
    pool.close()
    pool.join()
    
df['c'] = pd.concat(res.get(), axis=0, ignore_index=True)

In [ ]:
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(df.predicted_category)
labels = label_binarizer.classes_
print(labels)

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 150000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['c'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(df['c'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42)
x_train.shape, y_train.shape

# BiLSTM Model

In [ ]:
# model = Sequential()
# model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
# model.add(Bidirectional(LSTM(64)))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(rate=0.4))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(len(labels), activation="softmax"))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# epochs = 5
# batch_size = 128

# history = model.fit(x_train, y_train, epochs=epochs,
#                     batch_size=batch_size,validation_split=0.2,
#                    verbose =1
#                    )

In [ ]:
# def my_func(a):
#     n = a.argmax()
#     for i in range(len(a)):
#         if i==n:
#             a[i] = 1
#         else:
#             a[i] = 0
#     return a

# y_pred = model.predict(x_test)
# y_pr = np.apply_along_axis(my_func, 1, y_pred)
# print('accuracy score -->',accuracy_score(y_pr, y_test))      
# print('F1 score -->',f1_score(y_pr,y_test, average = 'weighted'))
# print('prescision score -->',precision_score(y_test, y_pr, average='macro'))
# print('recall score -->',recall_score(y_test, y_pr, average='macro'))
#classification_report(y_test, y_pr).split('\n')

In [ ]:
# %%time
# with multiprocessing.Manager().Pool(nthreads) as pool:
#     res = pool.map_async(cleanup_text, test['cleaned_hm'].values)
#     pool.close()
#     pool.join()
    
# test['c'] = pd.concat(res.get(), axis=0, ignore_index=True)
# X = tokenizer.texts_to_sequences(test['c'].values)
# X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
# print('Shape of data tensor:', X.shape)
# y_pred = model.predict(X)
# y_pred = np.apply_along_axis(my_func, 1, y_pred)
# test['predicted_category'] = label_binarizer.inverse_transform(y_pred, threshold=None)
# test[['hmid','predicted_category']].to_csv('submission.csv', index=False)

# Attention Model

In [ ]:
len_mat=[]
for i in range(len(X)):
    len_mat.append(len(X[i]))

In [ ]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
inputs=Input((X.shape[1],))
x=Embedding(input_dim=MAX_NB_WORDS+1,output_dim=EMBEDDING_DIM,input_length=X.shape[1],\
            embeddings_regularizer=keras.regularizers.l2(.001))(inputs)
att_in=Bidirectional(LSTM(64, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(x)
att_in = Dense(100, activation="relu")(att_in)
att_in = Dropout(0.4)(att_in)
att_in = Dense(100, activation="relu")(att_in)
att_out=attention()(att_in)
outputs=Dense(7,activation='softmax',trainable=True)(att_out)
model=Model(inputs,outputs)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
epochs = 16
batch_size = 128

history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,validation_split=0.2,
                   verbose =1
                   )

In [ ]:
def my_func(a):
    n = a.argmax()
    for i in range(len(a)):
        if i==n:
            a[i] = 1
        else:
            a[i] = 0
    return a

y_pred = model.predict(x_test)
y_pr = np.apply_along_axis(my_func, 1, y_pred)
print('accuracy score -->',accuracy_score(y_pr, y_test))      
print('F1 score -->',f1_score(y_pr,y_test, average = 'weighted'))
print('prescision score -->',precision_score(y_test, y_pr, average='macro'))
print('recall score -->',recall_score(y_test, y_pr, average='macro'))

In [ ]:
%%time
with multiprocessing.Manager().Pool(nthreads) as pool:
    res = pool.map_async(cleanup_text, test['cleaned_hm'].values)
    pool.close()
    pool.join()
    
test['c'] = pd.concat(res.get(), axis=0, ignore_index=True)
X = tokenizer.texts_to_sequences(test['c'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
y_pred = model.predict(X)
y_pred = np.apply_along_axis(my_func, 1, y_pred)
test['predicted_category'] = label_binarizer.inverse_transform(y_pred, threshold=None)
test[['hmid','predicted_category']].to_csv('submission_attn_8808.csv', index=False)